# 02_01

In [300]:
# 3rd pty modules
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from os import walk
from os.path import join

# Custom modules
from G_ERP_01_01 import get_xlsx, trim_None_col, trim_None_row

FromDir = 'Bom datas/Airblaster/AB trimed BOM'
ToDir = 'Bom datas/Airblaster/AB trimed head foot BOM'

## Function_1
### delete_head_foot_csv
* Delete the head and foot of the bom in single csv file

In [346]:
def delete_head_foot_csv(csv_file):
    # import dataFrame by path of the csv file
    bom = pd.read_csv(csv_file)
    list_row = []
    # delete the old index
    bom.drop(columns = 'Unnamed: 0', inplace = True)
    
    # loop through the rows of the dataFrame
    for row in bom.index:
        # delete the rows that contain cell with value less than 6 cells
        # by Airblaster's BOM it would find out the main area of the bom
        # As we've turned all cell lowercase, the None will be turned into a string "none"
        # Therefore, here we check nubmer of string 'none', to delete unwanted rows.
        if bom.iloc[row].str.contains('none').sum() > 3:
            list_row.append(row)
        elif bom.iloc[row].str.contains('update').sum() > 0:
            list_row.append(row)
        
    new_bom = bom.drop(list_row)
    
    # Add the "label" for the cell[0:1][0], 
    #it will be the first column header in the next step
    new_bom.iloc[0]['0'] = 'label' 
    
    # Double check if there any unwanted row.
    # in this moment, this column should be with name "none"
    # The column named 'none' should be last col if it exists.
    if new_bom.iloc[0].str.contains('none').sum() > 0:
        last_row = new_bom.shape[1]
        new_bom = new_bom.drop(new_bom.columns[last_row - 1], axis = 1)
            
    return new_bom

## Function_2
### delete_head_foot_dir
* Walk through a directory, the path(from_dir_path).
* Delete the head and foot of the bom in a csv that in the diretory
* Save all the trimed csv file to a directory, the path(to_dir_path).

In [347]:
def delete_head_foot_dir(from_dir_path, to_dir_path):
    for root, dirnames, filenames in walk(from_dir_path):
        
        # walk through each csv file
        for file_name in filenames:
            # get the path of the file
            # To Avoid the method to work with Mac original useless data.
            if file_name == '.DS_Store' or file_name.startswith('~$'):
                print('Detect unwanted file format')
            else:
                filepath = join(root, file_name)
                #Call the custom function to cut off the head and foot of the csv
                new_bom = delete_head_foot_csv(filepath)
                
                #Take first row as the Column Header
                new_bom.columns = new_bom.iloc[0]
                  
                #Delete all rows has same value as first row, 
                #as it still remain in the dataFrame
                new_bom = new_bom.iloc[pd.RangeIndex(len(new_bom)).drop(0)]

                #Save to the appointed directory
                new_bom.to_csv(join(to_dir_path, file_name))
                
    return

In [348]:
delete_head_foot_dir(FromDir, ToDir)
%time

Detect unwanted file format
CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.91 µs
